In [2]:
print(set('fred rodgers told me this'))
print(set('798654321')-set('92'))

{' ', 'f', 'g', 't', 'm', 'i', 'l', 's', 'o', 'h', 'e', 'r', 'd'}


In [3]:
# This is Jake's original code, commented out; I've replaced it below with
#   a very small modification where the index functions return sets; just to verify that works.
#
# Write functions that, given an index 0 <= i < 81,
# return the indices of grid spaces in the same row,
# column, and box as entry i
#
# def row_indices(i):
#     start = i - i % 9
#     return range(start, start + 9)
#
# def col_indices(i):
#     start = i % 9
#     return range(start, start + 81, 9)
#
# def box_indices(i):
#     start = 27 * (i // 27) + 3 * ((i % 9) // 3)
#     return [i for j in range(3) for i in range(start + 9 * j, start + 9 * j + 3)]
#
# compute and store the full set of connected indices for each i
# Notice that the three _indices(i) functions return lists of square indices...
#   These can have some degeneracy which is removed by the set() type cast since set elements are unique.
# The question I have is whether these set casts are necessary as the set.union() ought to do likewise. The
#   answer is yes (although I move these into the index functions) because union() operates on a tuple of sets;
#   it does not operate on lists.
# Subtracting out 'i' itself for a given i ensures that a given square is not considered one of the squares
#   that it is connected to.
# connected = [(set.union(set(box_indices(i)),
#                        set(row_indices(i)),
#                        set(col_indices(i)))
#               - set([i]))
#              for i in range(81)]

def row_indices(i):
    start = i - i % 9
    return set(range(start, start + 9))

def col_indices(i):
    start = i % 9
    return set(range(start, start + 81, 9))

def box_indices(i):
    start = 27 * (i // 27) + 3 * ((i % 9) // 3)
    return set([i for j in range(3) for i in range(start + 9 * j, start + 9 * j + 3)])

# Notice that because an integer is not iterable it cannot be passed as an argument to set()
connected = [(set.union(box_indices(i), row_indices(i), col_indices(i)) - set([i])) for i in range(81)]

# S(p) will recursively find solutions and "yield" them
def S(p):
    # First, find the number of empty squares and the number of
    # possible values within each square
    # L will be a list of tuples (triples) consisting of
    #   - the number of possible values for this square
    #   - the square's index
    #   - the set of those possible values
    L = []
    for i in range(81):
        if p[i] == '0':
            # vals = set('123456789') - set(p[n] for n in connected[i])
            vals = {'1', '2', '3', '4', '5', '6', '7', '8', '9'} - set(p[n] for n in connected[i])
            if len(vals) == 0: # At this square with value 0 = 'unsolved' there are no possible solutions
                return
            else:
                L.append((len(vals), i, vals))
    
    # if all squares are solved, then yield the current solution
    if len(L) == 0 and '0' not in p:
        yield p
        
    # otherwise, take the index with the smallest number of possibilities,
    # and recursively call S() for each possible value.
    else:
        N, i, vals = min(L)
        for val in vals:
            for s in S(p[:i] + val + p[i + 1:]):
                yield s

puz = "027800061000030008910005420500016030000970200070000096700000080006027000030480007"
puz2 = "647319258319258647258647913476582139921436875583971462165823794894765321732194586"

def test(S):
    # solve an empty puzzle
    print(next(S(81*'0')))
    for s in S(puz2): print(s)
    print('')

    # find all four solutions of puz
    for s in S(puz): print(s)

test(S)

425713698713698542698542137274185963931476285586329471159834726362957814847261359
647319258319258647258647913476582139921436875583971462165823794894765321732194586

327894561645132978918765423589216734463978215172543896794651382856327149231489657
327894561645132978918765423589216734463978215271543896794651382856327149132489657
327894561465132978918765423589216734643978215172543896794651382856327149231489657
327894561465132978918765423589216734643978215271543896794651382856327149132489657


In [16]:
# Write functions that, given an index 0 <= i < 81,
# return the indices of grid spaces in the same row,
# column, and box as entry i. 'sudoku-connected'
#
# A set is an unordered collection of unique and immutable objects... one of several 'collection' types in Python.
#   What is an example in Python of a mutable ordered collection? Answer: A list. Which is iterable.
#   What is an example in Python of an immutable ordered collection? Answer: A tuple. 
# Because sets are un-ordered (therefore not indexable) they do not contain duplicates. 
# 'connected' is in fact a list of sets, indexable by square 0, 1, ..., 80
# Let's track how this list of sets is constructed. 
#   First the outer square brackets ensure 'connected' is a list.
#   Second the contents of this list are built by iteration using the for-loop
#     The set() is given an iterable list via the square bracket.
#   Third 'connected' as a list has a (sudoku square) index from 0 to 80 inclusive by dint of 'i in range(81)
# Going further:
#   set([iterable list expression]) for i in range(81)
#     uses the for-loop structure to supply [list expression] with an iterated value i, as noted the square index.
#     The set() cast ensures that its internal list of connected squares [ j for j in etc ] has no duplicates
# Further still: What is the [ j for j in etc ] doing? Here the structure is
#   j for j in range(81) if <logical expression that uses j and i>
#     <logical> is true when j (enumerating over 0...81 square indices) matches at least one sudoku-connected conditions 
# English: The i'th element of 'connected' is the set of square indices that are sudoku-connected to square i. 
# Whew!
connected = [set(
                   [
                       j for j in range(81)
                       if (i%9==j%9)   or 
                          (i//9==j//9) or 
                          (i//27==j//27 and i%9//3==j%9//3)
                   ]
                )
                for i in range(81)]

# These print statements emphasize two important details: 
#   First that with our new version of connected just above: 'i' is no longer excluded from its own connected[i] entry
#   Second that the elements of the list connected[] are (immutable, un-ordered) sets
# if 4 in connected[4]: print('hey 4 is in 4!')
# print(connected[73])

# S(p) will recursively find solutions and "yield" them
def S(p):
    # First, find the number of empty squares and the number of
    # possible values within each square
    L = []
    for i in range(81):
        if p[i] == '0':
            vals = set('123456789') - set(p[n] for n in connected[i])
            if len(vals) == 0:
                return
            else:
                L.append((len(vals), i, vals))
    
    # if all squares are solved, then yield the current solution
    if len(L) == 0 and '0' not in p:
        yield p
        
    # otherwise, take the index with the smallest number of possibilities,
    # and recursively call S() for each possible value.
    else:
        N, i, vals = min(L)
        for val in vals:
            for s in S(p[:i] + val + p[i + 1:]):
                yield s

puz = "027800061000030008910005420500016030000970200070000096700000080006027000030480007"
puz2 = "647319258319258647258647913476582139921436875583971462165823794894765321732194586"

def test(S):
    # solve an empty puzzle
    print(next(S(81*'0')))
    for s in S(puz2): print(s)
    print('')

    # find all four solutions of puz
    for s in S(puz): print(s)

test(S)

647319258319258647258647913476582139921436875583971462165823794894765321732194586
647319258319258647258647913476582139921436875583971462165823794894765321732194586

327894561645132978918765423589216734463978215271543896794651382856327149132489657
327894561645132978918765423589216734463978215172543896794651382856327149231489657
327894561465132978918765423589216734643978215271543896794651382856327149132489657
327894561465132978918765423589216734643978215172543896794651382856327149231489657


In [13]:
# This version generates the 'connected' set on the fly at (only) square i; so i is not iterated
#   Note that in Python 3 19/9 = 2.1111111 whereas 19//9 is 2.
#   I swapped the if len(vals) since any non-zero will evaluate True
#   Simplified the yield of a solution as noticed in the blog comments

# S(p) will recursively find solutions and "yield" them
def S(p):
    L = []    # The list of 0-valued cells (i.e. un-resolved)
    for i in range(81):
        if p[i] == '0':
            vals = set('123456789') - \
                set(p[n] for n in set([j for j in range(81) if (i%9==j%9) or (i//9==j//9) or (i//27==j//27 and i%9//3==j%9//3)]))
            if len(vals): L.append((len(vals), i, vals))
            else: return
    
    # if all squares are resolved then the list of un-resolved squares will be empty; and so yield the current solution
    if len(L) == 0: yield p
        
    # otherwise, take the index with the smallest number of possibilities,
    # and recursively call S() for each possible value.
    else:
        N, i, vals = min(L)
        for val in vals:
            for s in S(p[:i] + val + p[i + 1:]): yield s

puz = "027800061000030008910005420500016030000970200070000096700000080006027000030480007"
puz2 = "647319258319258647258647913476582139921436875583971462165823794894765321732194586"

def test(S):
    # solve an empty puzzle
    print(next(S(81*'0')))   
    for s in S(puz2): print(s)
    print('')

    # find all four solutions of puz
    for s in S(puz): print(s)

test(S)

425713698713698542698542137274185963931476285586329471159834726362957814847261359
647319258319258647258647913476582139921436875583971462165823794894765321732194586

327894561645132978918765423589216734463978215172543896794651382856327149231489657
327894561645132978918765423589216734463978215271543896794651382856327149132489657
327894561465132978918765423589216734643978215172543896794651382856327149231489657
327894561465132978918765423589216734643978215271543896794651382856327149132489657


In [9]:
print(19//9)

2
